In [ ]:
import os
import glob
import subprocess
import shutil
from lxml import etree as ET

In [ ]:
def parse_a_structure_etree(source_folder, structure_etree):
    parent_pointer = os.path.split(source_folder)[1]
    collection_name = os.path.split(os.path.split(source_folder)[0])[1]
    if structure_etree.find('type').text == 'Monograph':
        print('monograph types are not supported: {}'.format(source_folder))
        return
    pointer_order = (i.find('pageptr').text for i in structure_etree.findall('page'))
    dest_path = '/media/francis/PNYusb3/cpd_to_pdfs/{}'.format(collection_name)
    os.makedirs(dest_path, exist_ok=True)
    
    source_mods = os.path.join(source_folder, 'MODS.xml')
    shutil.copy2(source_mods, os.path.join(dest_path, '{}.xml'.format(parent_pointer)))
    
    
    bash_script = ['img2pdf', '--output', os.path.join(dest_path, '{}.pdf'.format(parent_pointer))]
    bash_script.extend(os.path.join(source_folder, i,  "OBJ.jp2") for i in pointer_order)
    bash_execution = subprocess.run(bash_script)
    if bash_execution.returncode != 0:
        print('error in running img2pdf: \n{}'.format(bash_execution))

In [ ]:
parent_objects = set()

source_dir = '/home/francis/Desktop/lsu_git/islandora_ansible/UploadToIslandora/lsu-p16313coll31-cpd'

for root, dirs, files in os.walk(source_dir):
    if not dirs:
        continue
    if not os.path.split(root)[1].isnumeric():
        continue
    else:
        parent_objects.add(os.path.split(root)[1])
        
for root, dirs, files in os.walk(source_dir):
    this_parent = os.path.split(root)[1]
    if this_parent in parent_objects:
        structure_etree = ET.parse(os.path.join(root, 'structure.cpd'))
        parse_a_structure_etree(root, structure_etree)

In [ ]:
pdfs_dir = '/media/francis/PNYusb3/cpd_to_pdfs/lsu-p16313coll31-cpd/'
for root, dirs, files in os.walk(pdfs_dir):
    pointers_set = set(i.split('.')[0] for i in files)
    dest_dir = pdfs_dir.replace('cpd_to_pdfs', 'cpd_to_pdfs/zips')
    os.makedirs(dest_dir, exist_ok=True)
    for pointer in pointers_set:

        bash_command = ['zip',
                        '-0',
                        os.path.join(dest_dir, '{}.zip'.format(pointer))]
        bash_command.extend(glob.glob(os.path.join(root, '{}.*'.format(pointer))))
        print(' '.join(bash_command))
        print(subprocess.run(bash_command))